# Survey123 Choices to QR Codes
Dion Liddell, May 2020

## Overview
This notebook turns the Choices associated with a nominated Survey123 Dropdown Field into scannable QR Codes. Each QR Code generated represents one Choice in the Dropdown Field. Scan a generated QR Code with a modern smartphone camera, and the Survey will open in the smartphone's mobile web browser with the Dropdown Field correctly pre-populated.

The generated QR Codes could be used, for example, to help track building entry and exit movements.  Entry and Exit QR Codes could be placed on entranceways and exits: e.g. Libray Entry, Library Exit, Marketing Entry, Marketing Exit.

## Procedure
This Notebook was created and tested in ArcGIS Online - [ArcGIS Notebooks Beta](https://www.esri.com/arcgis-blog/products/arcgis-online/analytics/arcgis-notebooks-public-beta/)

1. Create a [Survey123 survey](https://survey123.arcgis.com/) that contains a nominated Dropdown Field that has been populated with a number of Choices
2. Take note of the (random) Survey ID contained within the Survey123 URL. e.g. ```https://survey123.arcgis.com/surveys/```**{my random Survey123 ID}**```/someparameter``` 
3. When publishing the survey, before clicking Publish for the second time, click Modify Schema, and take note of the Dropdown Field's **Name** (not the label)
4. Login into your ArcGIS Online organisation, and upload the *Survey123 Choices to QR Codes* notebook. Set the notebook's runtime to *ArcGIS Notebook Python 3 Standard (3.0)*
5. Open the notebook and update the *survey_id* and *dropdown_field* settings under the *Mandatory Settings to Modify* section to reflect your Survey ID and Dropdown Field Name recorded in the steps above
6. Click the Add button within the notebook, and click the hosted feature layer related to the survey (**not** the fieldworker layer). Copy the ID of the feature layer within the generated *get* method to the *survey_flc_id* under the *Mandatory Settings to Modify* cell
7. Delete the Cell from the notebook that was created in the step above (click within the cell and then select *Edit* and then *Delete Cells*)
8. Save the changes to the notebook
9. Select *Cell* in the notebook menu and then select *Run All*
10. Scoll down below the *Main* cell to see a list of the QR Codes that have been generated
11. Select *Files* from the top toolbar, and then click on the *home* folder, and then click on each of the generated QR Codes in turn to open them in a seperate browser tab or window
12. The QR Codes can be copied, pasted and printed.  Scan each with a smartphone camera to test

## References
https://community.esri.com/groups/survey123/blog/2019/02/06/survey123-tricks-of-the-trade-web-form-url-parameters

https://pypi.org/project/qrcode/

## Modules

In [ ]:
!pip install qrcode[pil]
import os, qrcode

from arcgis.gis import GIS
gis = GIS("home")

## Mandatory Settings to Modify

In [ ]:
# Mandatory to modify
survey_id = "replace with your survey ID"
dropdown_field = "replace with your dropdown field name"

survey_flc_id = "replace with your Survey123 hosted feature layer ID"

## Optional Settings to Modify

In [ ]:
# Optional to modify
qr_box_size = 20
home_dir = "/arcgis/home"
survey123_url = "https://survey123.arcgis.com/share"

## Main

In [ ]:
flc = gis.content.get(survey_flc_id)
fl = flc.layers[0]

# No error checking sorry, assuming that correct dropdown_field has been supplied
coded_values = next(_ for _ in fl.properties.fields if _.name == dropdown_field).domain.codedValues

for cv in coded_values:
    d = os.path.join(home_dir,cv.code + '.png') 
    if os.path.isfile(d):
        print(f"Deleting {d}")
        os.remove(d)
    u = f"{os.path.join(survey123_url,survey_id)}?field:{dropdown_field}={cv.code}"
    
    qr = qrcode.QRCode(
        version=3,
        error_correction=qrcode.constants.ERROR_CORRECT_M,
        box_size=qr_box_size,
        border=4
    )
    
    qr.add_data(u)
    qr.make(fit=True)
    img = qr.make_image()
    img.save(d)
    print(f"Created QR Code {d}\nfrom\n{u}\n\n")